# Propensity Scores

In [1]:
import pickle as pkl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics


KeyboardInterrupt



In [ ]:
data = pkl.load(open('../data/data.pkl', 'rb'))
data.head(2)

## Tests

In [ ]:
train_aucs, test_aucs = [], []
models = []
rocs = []
for i, (train_index, test_index) in enumerate(KFold(5).split(data)):
    train = data.iloc[train_index, :]
    test = data.iloc[test_index, :]
    x_train = train[train.columns[~train.columns.isin(['T','Y'])]]
    x_test = test[test.columns[~test.columns.isin(['T','Y'])]]
    t_train = train['T'].values
    t_test = test['T'].values
    
    # Logistic regression
    log_reg = LogisticRegression(max_iter=10000).fit(x_train, t_train)
    probs_train = log_reg.predict_proba(x_train)[:,1]
    probs_test = log_reg.predict_proba(x_test)[:, 1]
    test_fpr, test_tpr, _ = metrics.roc_curve(test['T'],  probs_test)
    test_auc = metrics.roc_auc_score(test['T'],  probs_test)
    train_fpr, train_tpr, _ = metrics.roc_curve(train['T'],  probs_train)
    train_auc = metrics.roc_auc_score(train['T'],  probs_train)
    models.append({'name': 'LR', 'epoch':i, 'train auc':train_auc, 'test auc':test_auc})
    rocs.append({'name': 'LR', 'epoch':i, 'test_fpr':test_fpr, 'test_tpr':test_tpr, 'train_fpr':train_fpr, 'train_tpr':train_tpr})
    
    # Random forest classifier
    rndfrst = RandomForestClassifier(max_depth=5, random_state=0).fit(x_train, t_train)
    probs_train = log_reg.predict(x_train)
    probs_test = log_reg.predict(x_test)
    test_fpr, test_tpr, _ = metrics.roc_curve(test['T'],  probs_test)
    test_auc = metrics.roc_auc_score(test['T'],  probs_test)
    train_fpr, train_tpr, _ = metrics.roc_curve(train['T'],  probs_train)
    train_auc = metrics.roc_auc_score(train['T'],  probs_train)
    models.append({'name': 'RF', 'epoch':i, 'train auc':train_auc, 'test auc':test_auc})
    rocs.append({'name': 'RF', 'epoch':i, 'test_fpr':test_fpr, 'test_tpr':test_tpr, 'train_fpr':train_fpr, 'train_tpr':train_tpr})
    
    # AdaBoost
    rndfrst = AdaBoostClassifier(n_estimators=100, random_state=0).fit(x_train, t_train)
    probs_train = log_reg.predict(x_train)
    probs_test = log_reg.predict(x_test)
    test_fpr, test_tpr, _ = metrics.roc_curve(test['T'],  probs_test)
    test_auc = metrics.roc_auc_score(test['T'],  probs_test)
    train_fpr, train_tpr, _ = metrics.roc_curve(train['T'],  probs_train)
    train_auc = metrics.roc_auc_score(train['T'],  probs_train)
    models.append({'name': 'AB', 'epoch':i, 'train auc':train_auc, 'test auc':test_auc})
    rocs.append({'name': 'AB', 'epoch':i, 'test_fpr':test_fpr, 'test_tpr':test_tpr, 'train_fpr':train_fpr, 'train_tpr':train_tpr})
                 

In [ ]:
models = pd.DataFrame(models)
models[['name', 'train auc', 'test auc']].groupby('name').mean()

In [ ]:
rocs = pd.DataFrame(rocs)

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(20,6))
epoch = 0
for i, name in enumerate(['LR', 'RF', 'AB']):
    ax = axes[i]
    alg_epoch = rocs[(rocs['epoch']==epoch)&(rocs['name']==name)]
    test_fpr, test_tpr = alg_epoch['test_fpr'].item(), alg_epoch['test_tpr'].item()
    train_fpr, train_tpr = alg_epoch['train_fpr'].item(), alg_epoch['train_tpr'].item()
    
    ax.plot(test_fpr, test_tpr, label=f'Test (AUC={round(test_auc,2)})')
    ax.plot(train_fpr, train_tpr, label=f'Train (AUC={round(train_auc,2)})')
    ax.set_ylabel('True Positive Rate')
    ax.set_xlabel('False Positive Rate')
    ax.legend()
    ax.set_title(f"ROC curve for {name}")
plt.show()

## Final calculation

In [ ]:
def calc_props(df):
    x = df[df.columns[~df.columns.isin(['T','Y'])]]
    t = df['T'].values
    log_reg = LogisticRegression(max_iter=10000).fit(x, t)
    probs = 
    df['propensity'] = log_reg.predict_proba(x)[:,1]

In [ ]:
data = calc_props(data)

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
data[data['T']==1].propensity.hist(bins=30, ax=ax, alpha=0.6, density=True, label='T = True')
data[data['T']==0].propensity.hist(bins=30, ax=ax, alpha=0.6, density=True, label='T = False')
plt.title("Propensity scores")
plt.legend()
plt.show()

In [ ]:
save = True
if save:
    pkl.dump(data, open("../data/data_p.pkl", "wb"))